**Pre-req:** Confirm that you are actually using a GPU instance by running `nvidia-smi` from terminal.  
**Important Note:** You will not see GPU being usable unless those modules have been loaded when the session was created.  

In [ ]:
# Env variable to optimize memory usage
import os
import datetime
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,preprocessing,Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import f1_score, confusion_matrix,precision_score,recall_score,accuracy_score,log_loss,roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display_html 

In [ ]:
# Define base path where files will be stored.
# This is unpacked from the pickle file created in Step 0.

with open('pickledHomeScratchShared.pickle', "rb") as f:
    baseHomePath,baseScratchPath,baseSharedPath = pickle.load(f)

In [ ]:
# Set memory growth on GPUs (another step for memory optimization)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print("Error: "+e)

#Print GPU devices, if any, that are available
physical_devices = tf.config.list_physical_devices('GPU')
print("Physical GPU Devices: ", physical_devices)

logical_devices = tf.config.list_logical_devices('GPU')
print("Logical GPU Devices: ", logical_devices)


## Section 1 : Setup re-usuable functions to be able to train each of CNN, Naive Bayes, Random Forest and Dummy Classifier


In [ ]:
# Code to help choose files based on file choice and view choice
def chooseFiles(viewChoice,fileChoice):
    if fileChoice == '608':
        transverseFile = 'skip_120_stitched_imgs_t_all_608.pickle'
        sagittalFile = 'skip_56_stitched_imgs_s_all_608.pickle'
        coronalFile = 'skip_120_stitched_imgs_c_all_608.pickle'
        labelsFile = 'all_labels_allImgs_608.pickle'
        mriIDFile = 'all_mri_id_allImgs_608.pickle'
    elif fileChoice == '809':
        transverseFile = 'skip_120_stitched_imgs_t_all_809.pickle'
        sagittalFile = 'skip_56_stitched_imgs_s_all_809.pickle'
        coronalFile = 'skip_120_stitched_imgs_c_all_809.pickle'
        labelsFile = 'all_labels_allImgs_809.pickle'
        mriIDFile = 'all_mri_id_allImgs_809.pickle'   
    elif fileChoice == '508':
        transverseFile = 'skip_120_stitched_imgs_t.pickle'
        sagittalFile = 'skip_56_stitched_imgs_s.pickle'
        coronalFile = 'skip_120_stitched_imgs_c.pickle'
        labelsFile = 'all_labels.pickle'
        mriIDFile = 'all_mri_id.pickle'   
    elif fileChoice in ['Processed','Pre-trained','Standard-ML-1D']:
        transverseFile = 'processed_img_t.pickle'
        sagittalFile = 'processed_img_s.pickle'
        coronalFile = 'processed_img_c.pickle'
        if viewChoice == 'Transverse':
            labelsFile = 'all_labels_processed_t.pickle'
            mriIDFile = 'all_mri_id_processed_t.pickle'
        elif viewChoice == 'Sagittal':
            labelsFile = 'all_labels_processed_s.pickle'
            mriIDFile = 'all_mri_id_processed_s.pickle'
        elif viewChoice == 'Coronal':
            labelsFile = 'all_labels_processed_c.pickle'
            mriIDFile = 'all_mri_id_processed_c.pickle'  
    # Load objects from serialized files
    if viewChoice == 'Transverse':
        with open("{}/{}".format(baseSharedPath,transverseFile), "rb") as f:
            img_16frames = pickle.load(f)
    elif viewChoice == 'Coronal':
        with open("{}/{}".format(baseSharedPath,coronalFile), "rb") as f:
            img_16frames = pickle.load(f)
    elif viewChoice == 'Sagittal':
        with open("{}/{}".format(baseSharedPath,sagittalFile), "rb") as f:
            img_16frames = pickle.load(f)
    with open("{}/{}".format(baseSharedPath,labelsFile), "rb") as f:
        all_labels = pickle.load(f)
    with open("{}/{}".format(baseSharedPath,mriIDFile), "rb") as f:
        all_mri_id = pickle.load(f)
        
    # 'Stack' the images  since it currently is represented in form of a scalar's shape (x,). Needs to be (x,y,z)
    img_16frames = np.stack(img_16frames, axis=0)

    if fileChoice == 'Pre-trained': # InceptionV3 needs 3 channels, hence have to adjust
        img_16frames = np.repeat(img_16frames[..., np.newaxis], 3, -1)
        print('Adjusting image shape for Pre-trained model to {}'.format(img_16frames.shape))
    elif fileChoice == 'Standard-ML-1D': # Flatten to 1-D for standard ML models E.g. Naive Bayes,Dummy Classifier,Random Forest
        img_16frames = img_16frames.reshape(-1, img_16frames.shape[1]*img_16frames.shape[2])
        print('Adjusting image shape for standard machine learning models to {}'.format(img_16frames.shape))
    else: # Reshape the images to 4 dimensional (1st dim is number of images, 2nd is height, 3rd is width and 4th is scalar)
        img_16frames = img_16frames.reshape(-1, img_16frames.shape[1], img_16frames.shape[2],1)

    return(img_16frames,all_labels,all_mri_id)

# Code to standardize dataset
def standardizeImg(img_16frames):
    # Flatten the array along the last dimension
    img_16frames_flat = img_16frames.reshape(-1, img_16frames.shape[-1])
    # Standardize the flattened array
    scaler = StandardScaler()
    img_16frames_flat_scaled = scaler.fit_transform(img_16frames_flat)
    # Reshape the standardized array to its original shape and reassign to 'img_16frames'
    img_16frames = img_16frames_flat_scaled.reshape(img_16frames.shape)
    return img_16frames


In [ ]:
# Re-usable code for generating best model using the hyperparamters found in sensitivity analysis
def generateBestCoronalModel():
    if viewChoice != 'Coronal':
        raise ValueError("This function is not meant for model generation for view other than Coronal ")
    model = models.Sequential()
    model.add(layers.Conv2D(filters=128, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.005), activation='relu'
                    ,input_shape=(img_16frames.shape[1], img_16frames.shape[2], 1), name = "C_2d_1"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.005), activation='relu', name = "C_2d_2"))
    model.add(layers.Dropout(0.4))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.005), activation='relu', name = "C_2d_3"))
    model.add(layers.Dropout(0.4))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size= 3
                    ,kernel_regularizer = tf.keras.regularizers.L2(0.005), activation='relu', name = "C_2d_4"))
    model.add(layers.Flatten())
    model.add(layers.Dense(16, kernel_regularizer = tf.keras.regularizers.L2(0.005), activation='relu', name = "Dense_1"))
    model.add(layers.Dense(2))
    tf.random.set_seed(seedValue)
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy']
                     )
    return model

def generateDummyClassifier():
    model = DummyClassifier(strategy='prior',random_state=seedValue) 
    return model

def generateNaiveBayes():
    model = GaussianNB()
    return model

def generateRandomForest():
    model = RandomForestClassifier(random_state=seedValue)
    return model

# Reusable code to generate f1_score, precision, recall and confusion matrix for a pair of X_test,y_test against a model
def generateF1Score(model,X_test,y_test):
    y_pred = model.predict(X_test)
    if fileChoice == 'Standard-ML-1D':
        f1Score = f1_score(y_test, y_pred, average='macro')
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        conf_matx = pd.DataFrame(confusion_matrix(y_test, y_pred), index = ['Actual-Neg', 'Actual-Pos'], columns = ['Pred-Neg', 'Pred-Pos'])
    else: # For 'Processed' , '809' etc.
        y_pred = y_pred.round(1)
        y_pred_binary = [0 if x[0] > x[1] else 1 for x in y_pred]
        y_test_binary = list(y_test.reshape(1,-1)[0])
        f1Score = f1_score(y_test_binary, y_pred_binary, average='macro')
        precision = precision_score(y_test_binary, y_pred_binary)
        recall = recall_score(y_test_binary, y_pred_binary)
        accuracy = accuracy_score(y_test_binary, y_pred_binary)
        conf_matx = pd.DataFrame(confusion_matrix(y_test_binary, y_pred_binary), index = ['Actual-Neg', 'Actual-Pos'], columns = ['Pred-Neg', 'Pred-Pos'])
    return (f1Score,precision,recall,accuracy,conf_matx)

def generateConfMatrix(CVScores,title=" "):
    conf_mat_list = [modelCVScore[-1] for modelCVScore in CVScores]
    i=0
    for conf_matx in conf_mat_list:
        i += 1
        conf_matx_styler = conf_matx.style.set_table_attributes("style='display:inline'").set_caption('{}--Iteration {}-{}'.format(title,i,'Coronal'))
        display_html(conf_matx_styler._repr_html_(), raw=True)
        
def generateScoreTable(CVScores,title=" "):
    modelDfC = pd.DataFrame([item[0:-1] for item in CVScores]
            ,columns= ["ActualNegativesInTrain" ,"ActualPositivesInTrain"
                              ,"ActualNegativesInTest", "ActualPositivesInTest"
                        ,"f1Score","precision","recall","accuracy"]
            ,index=['Iteration {}'.format(i) for i in range(1,n_splits+1)])
    modelDfC_styler = modelDfC.style.set_table_attributes("style='display:inline'").set_caption('{}'.format(title)).set_table_styles([{
                            'selector': 'caption',
                            'props': [('font-size', '20px')]
                        }])
    display(modelDfC_styler)
    return modelDfC

In [ ]:
# Choose seed value as 42 to be used
seedValue = 42
random.seed(seedValue) # Applies to whereever random library is used
print('Seed value used throughout this notebook is {}'.format(seedValue))
n_splits = 5
print('Number of k-folds applied on each experiment is : {}'.format(seedValue))

## Section 2 : Create best CNN model using the optimal values from Sensitivity analysis

In [ ]:
# File choices
# Best file to work with is 'Processed'
fileChoice = 'Processed'
print("Choice of file is : ",fileChoice)
# Best view is deemed to be 'Coronal' as seen from 809 vs Processed vs Pre-trained
viewChoice = 'Coronal'
print("Choice of view is : ",viewChoice)

In [ ]:
# Main code cell for training over stratified k-fold data for CNN

print('The choice of view is {}'.format(viewChoice))
print('The choice of file is {}'.format(fileChoice))
img_16frames,all_labels,all_mri_id = chooseFiles(viewChoice,fileChoice) # Generate dataset
img_16frames = standardizeImg(img_16frames) # Standardize dataset
CNNModelCVScores = [] # Empty list to store scores from each iteration
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
# Train for chosen view over k-folds using distributed strategy
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Iterate through each fold
    for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
        print('Iteration: {} for {}'.format(i+1,viewChoice))
        # Generate model from scratch 
        model = generateBestCoronalModel()
        checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                filepath=checkpoint_filepath, save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)
        ]
        # Generate new datasets for X_train, X_test, y_train, y_test on each run
        X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
        y_train, y_test = all_labels[train_index],all_labels[test_index]
        ActualNegativesInTrain ,ActualPositivesInTrain = len(y_train[y_train==0]),len(y_train[y_train==1])
        ActualNegativesInTest, ActualPositivesInTest= len(y_test[y_test==0]),len(y_test[y_test==1])
        model.fit(X_train,  y_train, epochs=20, 
                        validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
        history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
        # Store the evaluation metrics for this fold
        CNNModelCVScores.append([ActualNegativesInTrain ,ActualPositivesInTrain
                              ,ActualNegativesInTest, ActualPositivesInTest
                              ,*generateF1Score(model,X_test,y_test)])

In [ ]:
modelName = "Convolutional Neural Network"
generateConfMatrix(CNNModelCVScores,title="{}".format(modelName))
modelDF_CNN_Coronal = generateScoreTable(CNNModelCVScores,title="5-Fold Validation results for {}".format(modelName))

## Section 2 : Create best NaiveBayes, DummyClassifier and RandomForest model using default hyperparameter values to be able to compare.


In [ ]:
# File choices
# Best file to work with is 'Processed'
fileChoice = 'Standard-ML-1D'
print("Choice of file is : ",fileChoice)
# Best view is deemed to be 'Coronal' as seen from 809 vs Processed vs Pre-trained
viewChoice = 'Coronal'
print("Choice of view is : ",viewChoice)

In [ ]:
# Main code cell for training over stratified k-fold data for Dummy Classifier

print('The choice of view is {}'.format(viewChoice))
print('The choice of file is {}'.format(fileChoice))
img_16frames,all_labels,all_mri_id = chooseFiles(viewChoice,fileChoice) # Generate dataset
img_16frames = standardizeImg(img_16frames) # Standardize dataset
DummyModelCVScores = [] # Empty list to store scores from each iteration
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
# Train for chosen view over k-folds without using distributed strategy
#mirrored_strategy = tf.distribute.MirroredStrategy()
#with mirrored_strategy.scope():

# Iterate through each fold
for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
    print('Iteration: {} for {}'.format(i+1,viewChoice))
    # Generate model from scratch 
    model = generateDummyClassifier()
    # Generate new datasets for X_train, X_test, y_train, y_test on each run
    X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
    y_train, y_test = all_labels[train_index],all_labels[test_index]
    ActualNegativesInTrain ,ActualPositivesInTrain = len(y_train[y_train==0]),len(y_train[y_train==1])
    ActualNegativesInTest, ActualPositivesInTest= len(y_test[y_test==0]),len(y_test[y_test==1])
    model.fit(X_train,  y_train)
    # Store the evaluation metrics for this fold
    DummyModelCVScores.append([ActualNegativesInTrain ,ActualPositivesInTrain
                          ,ActualNegativesInTest, ActualPositivesInTest
                          ,*generateF1Score(model,X_test,y_test)])

In [ ]:
modelName = "Dummy Classifier"
generateConfMatrix(DummyModelCVScores,title="{}".format(modelName))
modelDF_Dummy_Coronal = generateScoreTable(DummyModelCVScores,title="5-Fold Validation results for {}".format(modelName))

In [ ]:
# Main code cell for training over stratified k-fold data for Naive Bayes

print('The choice of view is {}'.format(viewChoice))
print('The choice of file is {}'.format(fileChoice))
img_16frames,all_labels,all_mri_id = chooseFiles(viewChoice,fileChoice) # Generate dataset
img_16frames = standardizeImg(img_16frames) # Standardize dataset
NaiveBayesModelCVScores = [] # Empty list to store scores from each iteration
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
# Train for chosen view over k-folds without using distributed strategy
#mirrored_strategy = tf.distribute.MirroredStrategy()
#with mirrored_strategy.scope():

# Iterate through each fold
for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
    print('Iteration: {} for {}'.format(i+1,viewChoice))
    # Generate model from scratch 
    model = generateNaiveBayes()
    # Generate new datasets for X_train, X_test, y_train, y_test on each run
    X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
    y_train, y_test = all_labels[train_index],all_labels[test_index]
    ActualNegativesInTrain ,ActualPositivesInTrain = len(y_train[y_train==0]),len(y_train[y_train==1])
    ActualNegativesInTest, ActualPositivesInTest= len(y_test[y_test==0]),len(y_test[y_test==1])
    model.fit(X_train,  y_train)
    # Store the evaluation metrics for this fold
    NaiveBayesModelCVScores.append([ActualNegativesInTrain ,ActualPositivesInTrain
                          ,ActualNegativesInTest, ActualPositivesInTest
                          ,*generateF1Score(model,X_test,y_test)])

In [ ]:
modelName = "Naive Bayes"
generateConfMatrix(NaiveBayesModelCVScores,title="{}".format(modelName))
modelDF_NaiveBayes_Coronal = generateScoreTable(NaiveBayesModelCVScores,title="5-Fold Validation results for {}".format(modelName))

In [ ]:
# Main code cell for training over stratified k-fold data for Naive Bayes

print('The choice of view is {}'.format(viewChoice))
print('The choice of file is {}'.format(fileChoice))
img_16frames,all_labels,all_mri_id = chooseFiles(viewChoice,fileChoice) # Generate dataset
img_16frames = standardizeImg(img_16frames) # Standardize dataset
RandomForestModelCVScores = [] # Empty list to store scores from each iteration
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
# Train for chosen view over k-folds without using distributed strategy
#mirrored_strategy = tf.distribute.MirroredStrategy()
#with mirrored_strategy.scope():

# Iterate through each fold
for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
    print('Iteration: {} for {}'.format(i+1,viewChoice))
    # Generate model from scratch 
    model = generateRandomForest()
    # Generate new datasets for X_train, X_test, y_train, y_test on each run
    X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
    y_train, y_test = all_labels[train_index],all_labels[test_index]
    ActualNegativesInTrain ,ActualPositivesInTrain = len(y_train[y_train==0]),len(y_train[y_train==1])
    ActualNegativesInTest, ActualPositivesInTest= len(y_test[y_test==0]),len(y_test[y_test==1])
    model.fit(X_train,  y_train)
    # Store the evaluation metrics for this fold
    RandomForestModelCVScores.append([ActualNegativesInTrain ,ActualPositivesInTrain
                          ,ActualNegativesInTest, ActualPositivesInTest
                          ,*generateF1Score(model,X_test,y_test)])

In [ ]:
modelName = "Random Forest"
generateConfMatrix(RandomForestModelCVScores,title="{}".format(modelName))
modelDF_RandomForest_Coronal = generateScoreTable(RandomForestModelCVScores,title="5-Fold Validation results for {}".format(modelName))

In [ ]:
pd.DataFrame([modelDF_CNN_Coronal.mean(),modelDF_Dummy_Coronal.mean()
 ,modelDF_NaiveBayes_Coronal.mean(),modelDF_RandomForest_Coronal.mean()]
             ,index=['CNN','DummyClassifier','GaussianNaiveBayes','RandomForest'])

In [ ]:
cnn_vs_rest = {
'CNN' : CNNModelCVScores,
'DummyClassifier' : DummyModelCVScores,
'GaussianNaiveBayes' : NaiveBayesModelCVScores,
'RandomForest' : RandomForestModelCVScores
}

In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/cnn_vs_rest_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/model_scores',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(cnn_vs_rest, f)

## Learning curve for best model across different data sizes

In [ ]:
# Enter code here iterating over 50,100,150,200,250,300,350,400,436 samples to see effect

In [ ]:
# File choices
# Best file to work with is 'Processed'
fileChoice = 'Processed'
print("Choice of file is : ",fileChoice)
# Best view is deemed to be 'Coronal' as seen from 809 vs Processed vs Pre-trained
viewChoice = 'Coronal'
print("Choice of view is : ",viewChoice)

In [ ]:
# Main code cell for training over stratified k-fold data for CNN

print('The choice of view is {}'.format(viewChoice))
print('The choice of file is {}'.format(fileChoice))

CNNModelCVScores = [] # Empty list to store scores from each iteration
for size in [50,100,150,200,250,300,350,400,436]:
    img_16frames,all_labels,all_mri_id = chooseFiles(viewChoice,fileChoice) # Generate dataset
    img_16frames = standardizeImg(img_16frames) # Standardize dataset
    # Randomize order so that files chosen from truncating below does not have bias
    indices = np.arange(img_16frames.shape[0])
    np.random.shuffle(indices)
    img_16frames = img_16frames[indices]
    all_labels = all_labels[indices]
    # truncate file size here
    img_16frames = img_16frames[0:size,:,:,:]
    all_labels = all_labels[0:size]
    # Train using StratifiedKFold on the truncated
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seedValue) # Define the Stratified KFold object
    # Train for chosen view over k-folds using distributed strategy
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Iterate through each fold
        for i, (train_index, test_index) in enumerate(skf.split(img_16frames,all_labels)):
            print('Iteration: {} for {}'.format(i+1,viewChoice))
            # Generate model from scratch 
            model = generateBestCoronalModel()
            checkpoint_filepath = "{}/{}".format(os.path.dirname(baseSharedPath),'tmp') # 'tmp' folder in shared space
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint( # Saves best model on each epoch, which can be loaded and used
                    filepath=checkpoint_filepath, save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)
            ]
            # Generate new datasets for X_train, X_test, y_train, y_test on each run
            X_train, X_test =  img_16frames[train_index],img_16frames[test_index]
            y_train, y_test = all_labels[train_index],all_labels[test_index]
            ActualNegativesInTrain ,ActualPositivesInTrain = len(y_train[y_train==0]),len(y_train[y_train==1])
            ActualNegativesInTest, ActualPositivesInTest= len(y_test[y_test==0]),len(y_test[y_test==1])
            model.fit(X_train,  y_train, epochs=20, 
                            validation_data=(X_test, y_test),callbacks=callbacks,verbose=2)
            history = model.load_weights(checkpoint_filepath) # Load best model that is available from ModelCheckpoint callback data
            # Store the evaluation metrics for this fold
            CNNModelCVScores.append([i+1,size,ActualNegativesInTrain ,ActualPositivesInTrain
                          ,ActualNegativesInTest, ActualPositivesInTest,*generateF1Score(model,X_test,y_test)])


In [ ]:
# Pickle results with appropriate name so that it can be loaded and used if needed
runDateTime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
with open("{}/learning_curve_{}fold_seed{}_{}.pickle".format(baseSharedPath+'/model_scores',n_splits,seedValue,runDateTime), "wb") as f:
    pickle.dump(CNNModelCVScores, f)